# Get verbs from Rubinchik

In [1]:
import unicodedata as ud
from collections import defaultdict


class AlphabetDetector:
    def __init__(self, no_memory=False):
        self.alphabet_letters = defaultdict(dict)
        self.no_memory = no_memory

    def is_in_alphabet(self, uchr, alphabet):
        if self.no_memory:
            return alphabet in ud.name(uchr)
        try:
            return self.alphabet_letters[alphabet][uchr]
        except KeyError:
            return self.alphabet_letters[alphabet].setdefault(
                uchr, alphabet in ud.name(uchr))

    def only_alphabet_chars(self, unistr, alphabet):
        return all(self.is_in_alphabet(uchr, alphabet)
                   for uchr in unistr if uchr.isalpha())

    def detect_alphabet(self, unistr):
        return set(ud.name(char).split(' ')[0]
                   for char in unistr if char.isalpha())

    def is_cyrillic(self, unistr):
        return True if self.only_alphabet_chars(unistr, 'CYRILLIC') else False

    def is_latin(self, unistr):
        return True if self.only_alphabet_chars(unistr, 'LATIN') else False

    def is_arabic(self, unistr):
        return True if self.only_alphabet_chars(unistr, 'ARABIC') else False

In [2]:
ad = AlphabetDetector()


In [105]:
import re
persian = '[ا-یۀۀآـةيژؤإأ]+'
p = re.compile(persian)

#states = {(1,'word'), (2,'trans'), (3,'pos'), (4,'verb'), (5,'stem')}
auto = {}
AllPOS = set()
state = 0
word = ''
pos = ''
stem = ''

In [112]:

state = 0
Vocab = {}

def getPersian(line):
    match = p.search(line)
    if match and match.group() != None:
        return match.group().rstrip().strip('\u200c')
    else:
        print(line)
        return None

def addVerb(stem, verb):
    
    verb = verb.strip('\u200c')
    stem = stem.strip('\u200c')
    if (stem.strip().endswith('تن') or stem.strip().endswith('دن')) and (not verb.strip().endswith('تن') or not stem.strip().endswith('دن')):
        print(verb, stem)
        verbnew  = stem
        stem = verb
        verb = verbnew
        print(verb, stem)
    if verb in Vocab.keys() and Vocab[verb] != stem:
        Vocab[verb] = [Vocab[verb], stem]
        #print(str(verb[:-1])+'#'+str(stem))
        w.write(str(verb[:-1])+'#'+str(stem)+'\n')
    elif verb not in Vocab.keys():
        Vocab[verb] = stem
        #print(str(verb[:-1])+'#'+str(stem))
        w.write(str(verb[:-1])+'#'+str(stem)+'\n')

w = open('C:\\Users\\TK_adm\\Documents\\HSE\\comp_ling_progr\\iranica\\rubinchik\\verb-stem.txt', 'w', encoding='utf-8')
    
with open("C:\\Users\\TK_adm\\Documents\\HSE\\comp_ling_progr\\iranica\\rubinchik\\rubinchik.txt", 'r', encoding='utf-8') as f:
#with open("./rubinchik/rub_test.txt", 'r', encoding='utf-8') as f:
    for line in f:
        if ad.is_arabic(line.strip()) and state == 0: 
            state = 1
            #entry[0] = line.strip()
            word = line.strip().strip('\u200c')
            #print('state1')
        elif len(line)>2 and line.strip()[0] == '[' and line.strip()[-1] == ']' and state == 1:
            state = 2
            #print('state2')
            #print(line)
        elif ad.is_cyrillic(line.strip()) and state == 2:
            #print('state3')
            #if line.split('., ')[0].strip(' \\n') in poslist:
            state = 3
            #entry[1] = line.split('., ')[0].strip(' \\n')
            pos = line.split('., ')[0].strip(' \\n').strip('\\n')
            #AllPOS.add(pos)
            #if entry[1] == 'гл.':
            if 'гл.' in pos:
                #print('verb')
                state = 4
                #print('state4')
                
            #elif entry[1] == 'ОНВ':
            elif 'ОНВ' in pos:
                #print('ONV')
                state = 5
                #print('state5')
                
        elif state == 4 and 'ОНВ' in line:
            #print('verb --- ONV')
            if getPersian(line):
                stem = getPersian(line) 
            #print(stem)
            verb = word
            
            AllPOS.add(pos)
            #Vocab[entry[0]] = entry[1]
            #entry = verb, pos, stem
            #Vocab[verb] = stem
            #print(str(verb[:-1])+'#'+str(stem))
            addVerb(stem, verb)
            state = 0
            word = ''
            pos = ''
            stem = ''
            
        elif state == 5 and 'гл.' in line:
            #print('ONV ---- verb')
            if getPersian(line):
                verb = getPersian(line)
            stem = word
            #print(verb)
            
            AllPOS.add(pos)
            #Vocab[verb] = stem
            #print(str(verb[:-1])+'#'+str(stem))
            addVerb(stem, verb)
            state = 0
            word = ''
            pos = ''
            stem = ''
                #print(entry[1])
        elif line.startswith('ОНВ') and state == 2:
            if 'гл.' in line:
                #print('ONV+verb')
                if getPersian(line):
                    verb = getPersian(line)
                #print(verb)
                stem = word
                AllPOS.add(pos)
                #Vocab[verb] = stem
                addVerb(stem, verb)
                #print(str(verb[:-1])+'#'+str(stem))
                state = 0
                word = ''
                pos = ''
                stem = ''
        elif line.startswith('гл.') and state == 2:
            if 'ОНВ' in line:
                #print('verb+ONV')
                #parse
                if getPersian(line):
                    stem = getPersian(line)
                #print(stem)
                verb = word
                AllPOS.add(pos)
                #print(str(verb[:-1])+'#'+str(stem))
                #Vocab[verb] = stem
                addVerb(stem, verb)
                state = 0
                word = ''
                pos = ''
                stem = ''
            
        else:
            state = 0
            #print('state0')
            word = ''
            pos = ''
            stem = ''

w.close()

باش بودن
بودن باش
برافراز برافراشتن
برافراشتن برافراز
برانداز برانداختن
برانداختن برانداز
برشو برشدن
برشدن برشو
تنیدن تن
تن تنیدن
تندیدن تن
تن تندیدن
تنیدن تن
تن تنیدن
ОНВ гл.

سا سودن
سودن سا


In [110]:
len(Vocab)

349

In [63]:
len(AllPOS)
#AllPOS

405

In [113]:
#Vocab

# Get Nouns from Rubinchik

درجات
[darajāt]
сущ.
мн. от درجه

درجه
[daraje]
сущ.
мн. درجات [darajāt]

In [117]:
word = ''
noun = ''
plural = ''
state = 0
Nouns = {}

def getPersian(line):
    match = p.search(line)
    if match and match.group() != None:
        return match.group().rstrip().strip('\u200c')
    else:
        print(line)
        return None

def addNoun(noun, plural):
    
    noun = noun.strip('\u200c')
    plural = plural.strip('\u200c')
    
    if noun in Nouns.keys() and plural not in Nouns[noun]:

        Nouns[noun].append(plural)
        w.write(str(noun)+'#'+str(plural)+'\n')
    elif noun not in Nouns.keys():
        Nouns[noun] = []
        Nouns[noun].append(plural)
       
        w.write(str(noun)+'#'+str(plural)+'\n')

w = open('C:\\Users\\TK_adm\\Documents\\HSE\\comp_ling_progr\\iranica\\rubinchik\\noun-plural.txt', 'w', encoding='utf-8')
    
with open("C:\\Users\\TK_adm\\Documents\\HSE\\comp_ling_progr\\iranica\\rubinchik\\rubinchik.txt", 'r', encoding='utf-8') as f:

    for line in f:
        if ad.is_arabic(line.strip()) and state == 0: 
            state = 1
            
            word = line.strip().strip('\u200c')
            #print('state1')
        elif len(line) > 2 and line.strip()[0] == '[' and line.strip()[-1] == ']' and state == 1:
            state = 2
            #print('state2')
        elif ad.is_cyrillic(line.strip()) and state == 2 and 'сущ.' in line:
            #print('state3')
            state = 3
            #pos = line.split('., ')[0].strip(' \n').strip('\n')
        
            if 'мн. от' in line:
                #print('verb')
                if getPersian(line):
                    noun = getPersian(line) 
                plural = word
                addNoun(noun, plural)
                state = 0
                word = ''
                pos = ''
                noun = ''
                plural = ''
                #print('state4')
                
            elif 'мн.' in line:
                
                if getPersian(line):
                    plural = getPersian(line) 
                noun = word
                addNoun(noun, plural)
                state = 0
                word = ''
                pos = ''
                noun = ''
                plural = ''
                #print('state5')
                
        elif state == 3 and 'мн. от' in line:
            
            if getPersian(line):
                noun = getPersian(line) 
            
            plural = word
            addNoun(noun, plural)
            state = 0
            word = ''
            pos = ''
            noun = ''
            plural = ''
            
        elif state == 3 and 'мн.' in line:
            if getPersian(line):
                plural = getPersian(line)
            noun = word
            
            addNoun(noun, plural)
            state = 0
            word = ''
            pos = ''
            noun = ''
            plural = ''
               
        
            
        else:
            state = 0
            #print('state0')
            word = ''
            pos = ''
            stem = ''

w.close()

мн.

мн. дядья (дяди по отцу)

мн.

мн. ч. выдающиеся (от суффикса превосходной степени -tarin)

сущ. мн.

мн. ч. от 

мн. неслышащие, глухие



# Generate Noun with all affices

In [1]:
2 + 2

4

In [31]:


#conj + prep + stem + plur + pron + art + ra + cop
#conj + prep + stem + plur + pron + art + ke
#conj + prep + stem + plur + ez


conjs = {'و': 'CONJ ', '': ''}

preps = {'اندر': 'PREP ', 'در': 'PREP ', 'بر': 'PREP ', 'ز': 'PREP ', 'ب': 'PREP ', '': ''}

#stems = set() #get from file

#plur = ('ها', 'ان', 'جات', 'ات', 'گان', '')
#plurs = {'ها': 'PL', 'ان': 'PL', 'ات': 'PL', '': 'SG'}
plursE = {u'\u200c'+'ها': 'PL', u'\u200c'+'گان': 'PL', 'ات': 'PL', 'گان': 'PL', '': 'SG'}
plursAll = {'ها': 'PL', 'ان': 'PL', '': 'SG', u'\u200c'+'ها': 'PL'}


# u'\u200c' + pron
# stem[:-1] + pron if stem.endswith('ه') or stem.endswith('ة') and plur in {'جات', 'ات', 'گان'}

pronsAll = {'م': ' PRO1SG', 'ت': ' PRO2SG', 'ش': ' PRO3SG', 'مان': ' PRO1PL', 
         'تان': ' PRO2PL', 'شان': ' PRO3PL', '':'', u'\u200c'+'م': ' PRO1SG', u'\u200c'+'ت': ' PRO2SG', 
            u'\u200c'+'ش': ' PRO3SG', u'\u200c'+'مان': ' PRO1PL', 
         u'\u200c'+'تان': ' PRO2PL', u'\u200c'+'شان': ' PRO3PL'} 
pronsE = {u'\u200c'+'ام': ' PRO1SG', u'\u200c'+'ات': ' PRO2SG', u'\u200c'+'اش': ' PRO3SG',
          u'\u200c'+'امان': ' PRO1PL', 
         u'\u200c'+'اتان': ' PRO2PL', u'\u200c'+'اشان': ' PRO3PL', '':''}
pronsAIU = {'یم': ' PRO1SG', 'یت': ' PRO2SG', 'یش': ' PRO3SG', 'یمان': ' PRO1PL', 
         'یتان': ' PRO2PL', 'یشان': ' PRO3PL', '':''}

# u'\u200c' + pron
# 'ا' + pron if stem.endswith('ه')
# 'ی' + pron if stem.endswith('ی') or stem.endswith('و') or stem.endswith('ا')


artsAll = {'ی':' YEH', '': ''}
artsAIUE = {'یی': ' YEH', 'ئی': ' YEH', u'\u200c'+'یی': ' YEH', u'\u200c'+'ئی': ' YEH','': ''}
# art = 'یی' if stem.endswith('ی') or stem.endswith('و') or stem.endswith('ا') or stem.endswith('ه')
# art = 'ئی' if stem.endswith('ی') or stem.endswith('و') or stem.endswith('ا') or stem.endswith('ه')
# u'\u200c' + art

ras = {'را': ' POSTP', '': ''}


copsE = {u'\u200c'+'ام': ' COP1SG', u'\u200c'+'ای': ' COP2SG', u'\u200c'+'ست': ' COP3SG', 
         u'\u200c'+'ایم': ' COP1PL', u'\u200c'+'اید': ' COP2PL',u'\u200c'+'است': ' COP3SG',
       u'\u200c'+'اند': ' COP3PL', u'\u200c'+'م': ' COP1SG', u'\u200c'+'ی': ' COP2SG',  
         u'\u200c'+'یم': ' COP1PL', u'\u200c'+'ید': ' COP2PL', u'\u200c'+'ند': ' COP3PL', '': ''}

copsAll = {'م': ' COP1SG', 'ی': ' COP2SG', 'ست': ' COP3SG', 'یم': ' COP1PL', 'ید': ' COP2PL',
       'ند': ' COP3PL', u'\u200c'+'ام': ' COP1SG', u'\u200c'+'ای': ' COP2SG', u'\u200c'+'ست': ' COP3SG', 
         u'\u200c'+'ایم': ' COP1PL', u'\u200c'+'اید': ' COP2PL',u'\u200c'+'است': ' COP3SG',
       u'\u200c'+'اند': ' COP3PL', u'\u200c'+'م': ' COP1SG', u'\u200c'+'ی': ' COP2SG',  
         u'\u200c'+'یم': ' COP1PL', u'\u200c'+'ید': ' COP2PL', u'\u200c'+'ند': ' COP3PL', '': ''}
# u'\u200c' + cop
# 'ا' + cop #if stem.endswith('ه')
# u'\u200c' + cop

ezsAll = {'ی': ' EZ', '': ''}
ezsE = {u'\u200c'+'ی': ' EZ', '': ''}
# if stem.endswith('ی') or stem.endswith('و') or stem.endswith('ا') or stem.endswith('ه')
# u'\u200c' + ez if stem.endswith('ه')

kes = {'که': ' CONJ', '':''}  #if gloss.endswith('YEH')

print('done')

done


In [32]:
stems = {'کتاب'}

path = 'C:\\Users\\TK_adm\\Documents\\HSE\\comp_ling_progr\\iranica\\data\\'
#w = open(path+'Nforms.txt', 'w', encoding='utf-8')

forms = set()
k = 0

form = ''
tag = ''
for stem in stems:
    
    #form, tag = addAff(conjs, form, tag)
    
    for conj in conjs:
        conjform = ''
        conjtag = ''
        conjform += conj
        conjtag += conjs[conj]
        
        for prep in preps:
            prepform = ''
            preptag = ''
            prepform = conjform + prep
            preptag = conjtag + preps[prep]
            
            stemform = prepform + stem
            
            if (stemform.endswith('ه') or stemform.endswith('ة')):
                plurs = plursE
            else:
                plurs = plursAll
                
            for plur in plurs:
                
                plurform = ''
                plurtag = ''
                
                if plur == 'ات' or plur == 'گان':
                    stemform = stemform[:-1]
                
                plurform = stemform + plur
                plurtag = preptag + plurs[plur]
                
    
                if plurform.endswith('ی') or plurform.endswith('و') or plurform.endswith('ا'):
                    ezs = ezsAll
                elif (plurform.endswith('ه') or plurform.endswith('ة')):
                    ezs = ezsE
                else:
                    ezs = {'':''}
                # if stem.endswith('ی') or stem.endswith('و') or stem.endswith('ا') or stem.endswith('ه')
                # u'\u200c' + ez if stem.endswith('ه')

                for ez in ezs:
                    ezform = ''
                    eztag = ''
                    ezform = plurform + ez
                    eztag = plurtag + ezs[ez]
                    
                    forms.add((ezform, eztag))
                    k += 1
                    #if k%100 == 0:
                    #    print(k)
                    #if ezform:
                    #    w.write(ezform)
                    #    w.write('\t')
                    #    w.write(eztag)
                    #    w.write('\n')
                    
                
                if plurform.endswith('ی') or plurform.endswith('و') or plurform.endswith('ا'):
                    prons = pronsAIU
                elif (plurform.endswith('ه') or plurform.endswith('ة')):
                    prons = pronsE
                else:
                    prons = pronsAll
                
                for pron in prons:
                    pronform = ''
                    prontag = ''
                    pronform = plurform + pron
                    prontag = plurtag + prons[pron]
                    
                    if pronform.endswith('ی') or pronform.endswith('و') or pronform.endswith('ا') or pronform.endswith('ه') or pronform.endswith('ة'):
                        arts = artsAIUE
                    else:
                        arts = artsAll
                
                    for art in arts:
                        artform = ''
                        arttag = ''
                        artform = pronform + art
                        arttag = prontag + arts[art]
                        
                        if 'YEH' in arttag:
                            for ke in kes:
                                keform = ''
                                ketag = ''
                                keform = artform + ke
                                ketag = arttag + kes[ke]

                                forms.add((keform, ketag))
                                k += 1
                                #if k%100 == 0:
                                #    print(k)
                                #if keform:
                                #    w.write(keform)
                                #    w.write('\t')
                                #    w.write(ketag)
                                #    w.write('\n')
                                
                        
                        for ra in ras:
                            raform = ''
                            ratag = ''
                            
                            raform = artform + ra
                            ratag = arttag + ras[ra]
                            
                            if (raform.endswith('ه') or raform.endswith('ة')):
                                cops = copsE
                            else:
                                cops = copsAll
                                
                            for cop in cops:
                                copform = ''
                                coptag = ''
                                copform = raform + cop
                                coptag = ratag + cops[cop]
                                
                                forms.add((copform, coptag))
                                k += 1
                                #if k%100 == 0:
                                #    print(k)
                                #if copform:
                                #    w.write(copform)
                                #    w.write('\t')
                                #    w.write(coptag)
                                #    w.write('\n')
                                
                                #form = ''
                                #tag = ''
                        
                        
#w.close()
                
                
def addAff(affs, form, tag):
    for aff in affs:
        form += aff
        tag += affs[aff]
        
        return form, tag
    
    

In [33]:
len(forms)

39792

In [34]:
path = 'C:\\Users\\TK_adm\\Documents\\HSE\\comp_ling_progr\\iranica\\data\\'
with open(path+'Nforms.txt', 'w', encoding='utf-8') as w:
    for form, tag in forms:
        w.write(form)
        w.write('\t')
        w.write(tag)
        w.write('\n')
#forms

In [ ]:
na mi be present|past termPres|termPast|termPerf|termPP| pron

nas = {'': 'NEG', '': 'NEGMA', '': 'POS'}
mis = {'': 'MI', '':'MI', '':''}
bes = {'': 'BE', '': ''}

terms = {termPres, termPast, termPerf, termPP}
termPres = {'': ''}

termPP = {'': 'PP'}
prons = 

# Generate noun forms

In [163]:
w = open('C:\\Users\\TK_adm\\Documents\\HSE\\comp_ling_progr\\iranica\\rubinchik\\noun-forms.txt', 
         'w', encoding='utf-8')

Nouns = set()
Plurals = set()


def addSuff(sing, suffix):
    if sing[-1] == 'ه' and suffix == 'ها':
        return True
    if sing[-1] == 'ه' and suffix == 'ان':
        suffix = '\u200c' +  'گان'
        sing = sing[:-1]
    plur = sing + suffix
    plur_tagged = plur + '#' + sing + '#N+PL'
    if plur_tagged not in Nouns:
        Nouns.add(plur_tagged)
        w.write(plur_tagged+'\n') 
    
    sing_tagged = sing + '#' + sing + '#N+SG'
    if sing_tagged not in Nouns:
        Nouns.add(sing_tagged)
        w.write(sing_tagged+'\n')
         


with open('C:\\Users\\TK_adm\\Documents\\HSE\\comp_ling_progr\\iranica\\rubinchik\\noun-plural.txt', 
          'r', encoding='utf-8') as f:
    for line in f:
        sing, plur = line.strip('\n').split('#')
        
        if plur not in Plurals:
            Plurals.add(plur)
        sing_tagged = sing + '#' + sing + '#N+SG'
        if sing_tagged not in Nouns:
            Nouns.add(sing_tagged)
            w.write(sing_tagged+'\n')
        plur_tagged = plur + '#' + sing + '#N+PL'
        if plur_tagged not in Nouns:
            Nouns.add(plur_tagged)
            w.write(plur_tagged+'\n')

with open('C:\\Users\\TK_adm\\Documents\\HSE\\comp_ling_progr\\iranica\\rubinchik\\word-N.txt', 
          'r', encoding='utf-8') as f:    
    for line in f:
        sing = line.strip('\n')
        if sing in Plurals:
            print(sing)
            continue
            
        suf = '\u200c' + 'ها'
        suffices = ['ها', 'ان', suf]
        
        for suffix in suffices:
            if len(sing)>=1:
                addSuff(sing, suffix)
        
        
            
            
w.close()
        
        
    

آباء
آثار
آثام
آجال
آحاد
آذرع
آرا
آراء
آزادگان
آساد
آصار
آصال
آفات
آفاق
آلات
آلاف
آلام
آلهه
آموزندگان
آنات
آنیه
آیات
اباریق
اباعد
ابالسه
ابحاث
ابخره
ابخره
ابدان
ابراج
ابصار
ابصار
ابطال
ابطال
ابعاد
ابعاد
ابعار
ابعاض
ابکار
ابناء
ابنیه
ابواب
ابوال
ابیات
اتباع
اتباع
اتباع
اتباع
اتعاب
اتلال
اثقال
اثمار
اثمان
اثنیه
اثواب
اجانب
اجداد
اجراس
اجرام
اجرام
اجزاء
اجساد
اجسام
اجفان
اجلاد
اجلاف
اجله
اجناد
اجناس
اجنحه
اجنه
اجواف
اجوبه
اجور
اجهزه
اجیاد
اجیال
احادیث
احباء
احباب
احبار
احبال
احبه
احجار
احجام
احجام
احداث
احداث
احداق
احرار
احرام
احرام
احرف
احزاب
احزان
احساب
احقاب
احقاد
احکام
احکام
احلام
احمال
احوال
احوالات
احیان
اخبار
اخبار
اخدان
اخراج
اخراج
اخراج
اخراجات
اخراجات
اخریات
اخشاب
اخطار
اخطار
اخفاف
اخلاء
اخلاء
اخلاط
اخلاف
اخلاق
اخماس
اخوات
اخوان
اخیار
ادباء
ادراج
ادراج
ادراک
ادراک
ادعیه
ادله
ادناس
ادوات
ادوار
ادویه
ادویه‌جات
ادهار
ادهان
ادیار
ادیان
اذکار
اذکار
اذناب
اذواق
اذهان
اذیال
اراضی
اراقم
ارامل
ارامنه
ارانب
ارایک
ارباب
ارباح
ارباض
ارباع
ارباع
اربطه
ارتباطات
ارجا
ارجا
ارجاس
ارجل
ارحام
ارخت

# Get superlative adjectives

In [124]:
#'превосх. ст. от'


adj = ''
superl = ''
state = 0
Adjs = {}

def getPersian(line):
    match = p.search(line)
    if match and match.group() != None:
        return match.group().rstrip().strip('\u200c')
    else:
        print(line)
        return None

def addAdj(adj, superl):
    
    adj = adj.strip('\u200c')
    superl = superl.strip('\u200c')
    
    if adj in Adjs.keys() and superl not in Adjs[adj]:

        Adjs[adj].append(superl)
        w.write(str(adj)+'#'+str(superl)+'\n')
    elif adj not in Adjs.keys():
        Adjs[adj] = []
        Adjs[adj].append(superl)
       
        w.write(str(adj)+'#'+str(superl)+'\n')

w = open('C:\\Users\\TK_adm\\Documents\\HSE\\comp_ling_progr\\iranica\\rubinchik\\adj-superl.txt', 'w', encoding='utf-8')
    
with open("C:\\Users\\TK_adm\\Documents\\HSE\\comp_ling_progr\\iranica\\rubinchik\\rubinchik.txt", 'r', encoding='utf-8') as f:

    for line in f:
        if ad.is_arabic(line.strip()) and state == 0: 
            state = 1
            
            superl = line.strip().strip('\u200c')
            #print('state1')
        elif len(line) > 2 and line.strip()[0] == '[' and line.strip()[-1] == ']' and state == 1:
            state = 2
            #print('state2')
        elif ad.is_cyrillic(line.strip()) and state == 2 and 'прил' in line:
            #print('state3')
            state = 3
           
        elif state == 3 and 'превосх. ст. от' in line:
            
            if getPersian(line):
                adj = getPersian(line) 
            
            
            addAdj(adj, superl)
            state = 0
            adj = ''
            superl = ''
            
        else:
            state = 0
            #print('state0')
            adj = ''
            superl = ''     

w.close()

# Generate adjective forms

In [165]:
w = open('C:\\Users\\TK_adm\\Documents\\HSE\\comp_ling_progr\\iranica\\rubinchik\\adj-forms.txt', 
         'w', encoding='utf-8')

Adjs = set()
Plurals = set()


def addSuff(sing, suffix, tag):
    if sing[-1] == 'ه':
        suffix = '\u200c' + suffix
    form_tagged = sing + suffix +'#' + sing + '#ADJ+'+tag
    if form_tagged not in Adjs:
        w.write(form_tagged+'\n')
        Adjs.add(form_tagged)
    sing_tagged = sing +'#' + sing + '#ADJ+POS'
    if sing_tagged not in Adjs:
        w.write(sing_tagged+'\n')
        Adjs.add(sing_tagged)

         


with open('C:\\Users\\TK_adm\\Documents\\HSE\\comp_ling_progr\\iranica\\rubinchik\\adj-superl.txt', 
          'r', encoding='utf-8') as f:
    for line in f:
        adj, superl = line.strip('\n').split('#')
        
        #if plur not in Plurals:
        #    Plurals.add(plur)
        adj_tagged = adj + '#' + adj + '#ADJ+POS'
        if adj_tagged not in Adjs:
            Adjs.add(adj_tagged)
            w.write(adj_tagged+'\n')
        superl_tagged = superl + '#' + adj + '#ADJ+SUPER+AR'
        if superl_tagged not in Adjs:
            Adjs.add(superl_tagged)
            w.write(superl_tagged+'\n')

with open('C:\\Users\\TK_adm\\Documents\\HSE\\comp_ling_progr\\iranica\\rubinchik\\word-ADJ.txt', 
          'r', encoding='utf-8') as f:    
    for line in f:
        adj = line.strip('\n')
        #if adj in Plurals:
        #    print(sing)
        #    continue
            
        comp = 'تر'
        sup = 'ترین'
        
        if len(adj)>=1:
            addSuff(adj, comp, 'COMP')
            addSuff(adj, sup, 'SUPER')

        
            
            
w.close()
        

# SIMPLE. Get words from Rubinchik

In [10]:
#states = {(1,'word'), (2,'trans'), (3,'pos')}
auto = {}
pos = set()
#poslist = 
state = 0
word = ['word', 'pos']
Vocab = {}

with open("./rubinchik/rubinchik.txt", 'r', encoding='utf-8') as f:
    for line in f:
        if ad.is_arabic(line.strip()) and state == 0: 
            state = 1
            word[0] = line.strip()
            #print(line)
        elif line.strip()[0] == '[' and line.strip()[-1] == ']' and state == 1:
            state = 2
            #print(line)
        elif ad.is_cyrillic(line.strip()) and state == 2: 
            #if line.split('., ')[0].strip(' \\n') in poslist:
            state = 3
            word[1] = line.split('., ')[0].strip(' \\n')
            pos.add(word[1])
            Vocab[word[0]] = word[1]
            state = 0   
                #print(word[1])
        else:
            state = 0

In [12]:
#pos
with open('./rubinchik/rub_wordlist.txt', 'w', encoding='utf-8') as w:
    dicSorted = sorted(Vocab.items(), key=lambda tup: tup[0], reverse=False)
    for word in dicSorted:
        w.write(word[0]+'\t'+str(word[1])+'\n')
        #i+=1 

# Get full words with POS from Rubinchik

In [153]:
#states = {(1,'word'), (2,'trans'), (3,'pos')}
auto = {}
allPOS = set()
#poslist = 
state = 0
word = ''
pos = ''
WORDS = {}
Other = {}

POSlist = ('N','ADJ','V','ADV','NUM','PRO','PP','P','CON','PART','PARTC','NOM','INT','MOD','PS','VVOD')

w = open('C:\\Users\\TK_adm\\Documents\\HSE\\comp_ling_progr\\iranica\\rubinchik\\word-pos.txt', 'w', encoding='utf-8')
wn = open('C:\\Users\\TK_adm\\Documents\\HSE\\comp_ling_progr\\iranica\\rubinchik\\word-N.txt', 'w', encoding='utf-8')
wa = open('C:\\Users\\TK_adm\\Documents\\HSE\\comp_ling_progr\\iranica\\rubinchik\\word-ADJ.txt', 'w', encoding='utf-8')
wv = open('C:\\Users\\TK_adm\\Documents\\HSE\\comp_ling_progr\\iranica\\rubinchik\\word-V.txt', 'w', encoding='utf-8')
wr = open('C:\\Users\\TK_adm\\Documents\\HSE\\comp_ling_progr\\iranica\\rubinchik\\word-pos-unchange.txt', 'w', encoding='utf-8')


def addWord(word, pos):
    word = word.strip('\u200c').strip('\n')
    #pos = pos.strip('\u200c').strip('\n')
    allPOS.add(pos)
    if word in WORDS.keys():# and pos not in Nouns[noun]:    
        WORDS[word].append(pos)
        w.write(str(word)+'#'+str(pos)+'\n')
    #elif noun not in Nouns.keys():
    else:
        #Nouns[noun] = []
        #Nouns[noun].append(plural)
        WORDS[word] = []
        WORDS[word].append(pos)
        w.write(str(word)+'#'+str(pos)+'\n')
        
    if pos == 'N':
        wn.write(str(word)+'\n')
    elif pos == 'V':
        wv.write(str(word)+'\n')
    elif pos == 'ADJ':
        wa.write(str(word)+'\n')
    else:
        wr.write(str(word)+'#'+str(pos)+'\n')

with open("C:\\Users\\TK_adm\\Documents\\HSE\\comp_ling_progr\\iranica\\rubinchik\\rubinchik.txt", 'r', encoding='utf-8') as f:
    for line in f:
        if ad.is_arabic(line.strip()) and state == 0: 
            state = 1
            word = line.strip()
            #print(line)
        elif len(line) > 2 and line.strip()[0] == '[' and line.strip()[-1] == ']' and state == 1:
            state = 2
            #print(line)
        elif ad.is_cyrillic(line.strip()) and state == 2: 
            
            pos = line.split('., ')[0].strip(' \n').strip('\n')
            
            #if line.split('., ')[0].strip(' \\n') in poslist:
            state = 3
            if pos.startswith('сущ'):
                pos = 'N'
            elif pos.startswith('прил'):
                pos = 'ADJ'
            elif pos.startswith('гл'):
                pos = 'V'
            elif pos.startswith('нар'):
                pos = 'ADV'
            elif pos.startswith('числ'):
                pos = 'NUM'
            elif pos.startswith('мест'):
                pos = 'PRO'
            elif pos.startswith('предл. отым.'):
                pos = 'PP'
            elif pos.startswith('предл'):
                pos = 'P'
            elif pos.startswith('союз'):
                pos = 'CON'  
            elif pos.startswith('част'):
                pos = 'PART' 
            elif pos.startswith('прич'):
                pos = 'PARTC'
            elif pos.startswith('имя'):
                pos = 'NOM'
            elif pos.startswith('межд'):
                pos = 'INT'
            elif pos.startswith('мод. сл.'):
                pos = 'MOD'
            elif pos.startswith('сл.-предл.'):
                pos = 'PS'
            elif pos.startswith('вводн'):
                pos = 'VVOD'
            elif 'прил' in line:
                pos = 'ADJ'
            elif 'гл' in line:
                pos = 'V'
            elif 'нар' in line:
                pos = 'ADV'
            elif 'числ' in line:
                pos = 'NUM'
            elif 'мест' in line:
                pos = 'PRO'
            elif 'предл. отым.' in line:
                pos = 'PP'
            elif 'предл' in line:
                pos = 'P'
            elif 'союз' in line:
                pos = 'CON'  
            elif 'част' in line:
                pos = 'PART' 
            elif 'прич' in line:
                pos = 'PARTC'
            elif 'имя' in line:
                pos = 'NOM'
            elif 'межд' in line:
                pos = 'INT'
            elif 'мод. сл.' in line:
                pos = 'MOD'
            elif 'сл.-предл.' in line:
                pos = 'PS'
            elif 'вводн' in line:
                pos = 'VVOD'
            else:
                Other[word] = line
            
            
            #allPOS.add(pos)
            if pos in POSlist:
                addWord(word, pos)
            state = 0   
                #print(word[1])
        else:
            state = 0
            
w.close()


 #'дееприч','предикат.',


AttributeError: 'str' object has no attribute 'close'

In [140]:
len(allPOS)
allPOS

{'ADJ',
 'ADV',
 'CON',
 'INT',
 'MOD',
 'N',
 'NOM',
 'NUM',
 'P',
 'PART',
 'PARTC',
 'PP',
 'PRO',
 'PS',
 'V',
 'VVOD'}

In [141]:
len(WORDS)

52201

In [142]:
len(Other)
#Other

582

In [143]:
multi = [w for w in WORDS.keys() if len(WORDS[w]) > 1]
len(multi)

5969

# Write single-pos words and separate words with |

In [150]:
single = [w for w in WORDS.keys() if len(WORDS[w]) == 1]
len(single)

46232

In [152]:
def check4multi(word):
    back = []
    if '|' not in word:
        return False
    else:
        s = word
        if '\u200c' in word:
            s = word.replace('\u200c', '$')
        words = s.split('|')
        
        for w in words:
            new = w.replace('$', '\u200c')
            back.append(new)
        return back




f = open('C:\\Users\\TK_adm\\Documents\\HSE\\comp_ling_progr\\iranica\\rubinchik\\word-pos-single-sep.txt', 'w', encoding='utf-8')
for w in single:
    if not check4multi(w):
        f.write(str(w)+'#')
        f.write(WORDS[w][0]+'\n')
    else:
        for word in check4multi(w):
            f.write(str(word)+'#')
            f.write(WORDS[w][0]+'\n')
f.close()

In [148]:
print(multi[1000], WORDS[multi[1000]])

قیم ['N', 'N']


# Write multi-pos words and separate words with |

In [ ]:
multi = [w for w in WORDS.keys() if len(WORDS[w]) > 1]
len(multi)

In [151]:
def check4multi(word):
    back = []
    if '|' not in word:
        return False
    else:
        s = word
        if '\u200c' in word:
            s = word.replace('\u200c', '$')
        words = s.split('|')
        
        for w in words:
            new = w.replace('$', '\u200c')
            back.append(new)
        return back




f = open('C:\\Users\\TK_adm\\Documents\\HSE\\comp_ling_progr\\iranica\\rubinchik\\word-pos-multi_sep.txt', 'w', encoding='utf-8')
for w in multi:
    if not check4multi(w):
        f.write(str(w)+'#')
        for pos in WORDS[w][:-1]:
            f.write(pos+'-')
        f.write(WORDS[w][-1]+'\n')
    else:
        for word in check4multi(w):
            f.write(str(word)+'#')
            for pos in WORDS[w][:-1]:
                f.write(pos+'-')
            f.write(WORDS[w][-1]+'\n')
f.close()

In [ ]:
#ОНВ آزار‌ [āzār]
#ОНВ آزما(ی) [āzmā(y)]
#гл. ОНВ آژن [āžan]
#ОНВ гл. آشکوخ [āškux] и اشکوخ [aškux]
#1) ОНВ гл. آفریدن

In [3]:
#from __future__ import unicode_literals
#import hazm
#normalizer = Normalizer()
#normalizer.normalize('اصلاح نويسه ها و استفاده از نیم‌فاصله پردازش را آسان مي كند')


from __future__ import unicode_literals
import re, codecs
from .utils import default_verbs
from nltk.tokenize.api import TokenizerI


class WordTokenizer(TokenizerI):
	"""
	>>> tokenizer = WordTokenizer()
	>>> tokenizer.tokenize('این جمله (خیلی) پیچیده نیست!!!')
	['این', 'جمله', '(', 'خیلی', ')', 'پیچیده', 'نیست', '!!!']
	"""

	def __init__(self, verbs_file=default_verbs, join_verb_parts=True):
		self._join_verb_parts = join_verb_parts
		self.pattern = re.compile(r'([؟!\?]+|[:\.،؛»\]\)\}"«\[\(\{])')

		if join_verb_parts:
			self.after_verbs = set([
				'ام', 'ای', 'است', 'ایم', 'اید', 'اند', 'بودم', 'بودی', 'بود', 'بودیم', 'بودید', 'بودند', 'باشم', 'باشی', 'باشد', 'باشیم', 'باشید', 'باشند',
				'شده_ام', 'شده_ای', 'شده_است', 'شده_ایم', 'شده_اید', 'شده_اند', 'شده_بودم', 'شده_بودی', 'شده_بود', 'شده_بودیم', 'شده_بودید', 'شده_بودند', 'شده_باشم', 'شده_باشی', 'شده_باشد', 'شده_باشیم', 'شده_باشید', 'شده_باشند',
				'نشده_ام', 'نشده_ای', 'نشده_است', 'نشده_ایم', 'نشده_اید', 'نشده_اند', 'نشده_بودم', 'نشده_بودی', 'نشده_بود', 'نشده_بودیم', 'نشده_بودید', 'نشده_بودند', 'نشده_باشم', 'نشده_باشی', 'نشده_باشد', 'نشده_باشیم', 'نشده_باشید', 'نشده_باشند',
				'شوم', 'شوی', 'شود', 'شویم', 'شوید', 'شوند', 'شدم', 'شدی', 'شد', 'شدیم', 'شدید', 'شدند',
				'نشوم', 'نشوی', 'نشود', 'نشویم', 'نشوید', 'نشوند', 'نشدم', 'نشدی', 'نشد', 'نشدیم', 'نشدید', 'نشدند',
				'می‌شوم', 'می‌شوی', 'می‌شود', 'می‌شویم', 'می‌شوید', 'می‌شوند', 'می‌شدم', 'می‌شدی', 'می‌شد', 'می‌شدیم', 'می‌شدید', 'می‌شدند',
				'نمی‌شوم', 'نمی‌شوی', 'نمی‌شود', 'نمی‌شویم', 'نمی‌شوید', 'نمی‌شوند', 'نمی‌شدم', 'نمی‌شدی', 'نمی‌شد', 'نمی‌شدیم', 'نمی‌شدید', 'نمی‌شدند',
				'خواهم_شد', 'خواهی_شد', 'خواهد_شد', 'خواهیم_شد', 'خواهید_شد', 'خواهند_شد',
				'نخواهم_شد', 'نخواهی_شد', 'نخواهد_شد', 'نخواهیم_شد', 'نخواهید_شد', 'نخواهند_شد',
			])

			self.before_verbs = set([
				'خواهم', 'خواهی', 'خواهد', 'خواهیم', 'خواهید', 'خواهند',
				'نخواهم', 'نخواهی', 'نخواهد', 'نخواهیم', 'نخواهید', 'نخواهند'
			])

			with codecs.open(verbs_file, encoding='utf8') as verbs_file:
				self.verbs = list(reversed([verb.strip() for verb in verbs_file if verb]))
				self.bons = set([verb.split('#')[0] for verb in self.verbs])
				self.verbe = set([bon +'ه' for bon in self.bons] + ['ن'+ bon +'ه' for bon in self.bons])

	def tokenize(self, text):
		text = self.pattern.sub(r' \1 ', text.replace('\n', ' ').replace('\t', ' '))
		tokens = [word for word in text.split(' ') if word]
		if self._join_verb_parts:
			tokens = self.join_verb_parts(tokens)
		return tokens

	def join_verb_parts(self, tokens):
		"""
		>>> tokenizer = WordTokenizer()
		>>> tokenizer.join_verb_parts(['خواهد', 'رفت'])
		['خواهد_رفت']
		>>> tokenizer.join_verb_parts(['رفته', 'است'])
		['رفته_است']
		>>> tokenizer.join_verb_parts(['گفته', 'شده', 'است'])
		['گفته_شده_است']
		>>> tokenizer.join_verb_parts(['گفته', 'خواهد', 'شد'])
		['گفته_خواهد_شد']
		>>> tokenizer.join_verb_parts(['خسته', 'شدید'])
		['خسته', 'شدید']
		"""

		result = ['']
		for token in reversed(tokens):
			if token in self.before_verbs or (result[-1] in self.after_verbs and token in self.verbe):
				result[-1] = token +'_'+ result[-1]
			else:
				result.append(token)
		return list(reversed(result[1:]))

SystemError: Parent module '' not loaded, cannot perform relative import

In [20]:
path = r"C:\\Users\\TK_adm\\Documents\\HSE\\comp_ling_progr\\iranica\\collect\\poems\\"
punct = {'،', '«', '»', ':', '!', '؟', '؛','...', '.', '(',')'}

Lemmas = {}

with open(path+'corpus_big.txt', 'r', encoding='utf-8') as f:
    for line in f:
        words = line.split()
        for w in words:
            word = w
            pre, post = '',''
            #haypunct = True
            tokens = []

            if len(word) and word[0] in punct: 
                pre = word[0]
                word = word[1:]
                tokens.append(pre)
            if len(word) and word[-1] in punct:
                post = word[-1]
                word = word[:-1]
                tokens.append(post)
            #if len(word) and word[0] not in punct and word[-1] not in punct:
                #haypunct = False
                #tokens.append(word)
            tokens.append(word)
        
            for token in tokens:
                if token in Lemmas:
                    Lemmas[token] += 1
                else:
                    Lemmas[token] = 1
        
        
with open(path+'frequency_punct1.txt', 'w', encoding='utf-8') as w:
    dicSorted = sorted(Lemmas.items(), key=lambda tup: tup[1], reverse=True)
    for word in dicSorted:
        w.write(word[0]+'\t'+str(word[1])+'\n')
        #i+=1 

In [21]:
len(Lemmas)

36894

In [22]:
len(Vocab)

52612

In [ ]:
'''
with open(path+'corpus_big.txt', 'r', encoding='utf-8') as f:
    for line in f:
        words = line.split()
        for w in words:
            word = w
            pre, post = '',''
            haypunct = True
            tokens = []
            while haypunct:
                if len(word) and word[0] in punct: 
                    pre = word[0]
                    word = word[1:]
                    tokens.append(pre)
                if len(word) and word[-1] in punct:
                    post = word[-1]
                    word = word[:-1]
                    tokens.append(post)
                if len(word) and word[0] not in punct and word[-1] not in punct:
                    haypunct = False
                    tokens.append(word)
                    
                    for token in tokens:
                        if token in Lemmas:
                            Lemmas[token] += 1
                        else:
                            Lemmas[token] = 1
'''

In [ ]:
with open(path+'frequency_punct_alpha.txt', 'w', encoding='utf-8') as w:
    dicSorted = sorted(Lemmas.items(), key=lambda tup: tup[0], reverse=False)
    for word in dicSorted:
        w.write(word[0]+'\n')
        #i+=1 

In [32]:
Corpus = set()
with open('BijanKhan_Distinct_Words.txt','r', encoding='utf-8') as corpus:
    for line in corpus:
        Corpus.add(line)

In [33]:
len(Corpus)

76707

In [38]:
with open(path+'corp_rub_corpus_dif.txt', 'w', encoding='utf-8') as f:
    for w in Lemmas:
        if w in Vocab:
            continue 
        if w in Corpus:
            continue
        f.write(w+'\n')

In [49]:
pre = {'ز','ب','و'}
post = {'ست','م','ت','ش'}

with open(path+'corp_rub_corpus_dif_cut.txt', 'w', encoding='utf-8') as f:
    for w in Lemmas:
        if w in Vocab:
            continue 
        if w in Corpus:
            continue
        word = w
        if len(word) and word[0] in pre:
            if len(word) and (word[1:] in Vocab or word[1:] in Corpus): 
                if Vocab[word[1:]] == 'гл.\n':
                    print(word)
                else:
                    f.write(word[0]+'\t'+word[1:]+'\n')
        if len(word) and word[-1] in post:
            if len(word) and (word[:-1] in Vocab or word[:-1] in Corpus):  
                    f.write(word[:-1]+'\t'+word[-1]+'\n')
                

بباشد
بدادن
بدرودن
بنوشتن
بگرفتیم
بگداختن
بگشادن
باختن
ببریدن
برفتن
بگریختن
بدیدن
بگفتن
بنبشتن
بپرداختن


In [ ]:
#27148

In [43]:
Vocab['افتادن']

'гл.\n'